# ChatMyArchive

Hao Li, Sacha

# Motivation: something that works, can be verified, on private sources,

 - search for law files, lots of pdf, special structure, wording/phrasing, Claude refusing, GPT doesn't answer
 - 1. want to be **correct, referenced for very sure (otherwise get sued for publishing fake law/ news or plagarism, I mean, the AI does not get prisoned, users, could)** Good guesses are good, but this is law journalism, serious content, answers **can be validified by people** 2. but people can only read finite content **when we validate it, we don't spend even more time**
 - run locally, accessibility, privacy, customizable


# Yesterday

 - at least we found Kim Kurdashian
 - splitter: too short, not useful; too long? not accurate & wasteful
 - modifying RAG,tweaking, local LLM

# Last night:
 - sort of? HPO of RAG & searching with metadata
 - How? **Mixtral 8x7b LOCALLY**, **Modified Retriever (retrieve+ expand metadata)**
F
# Today

- "trying everything to search?" RAG as tool, tool as RAG, running multiple QA sequences.
- Someone has to use it. (pip install?, minimum of UI, import/ export from spreadsheet): idea: researching vs re-searching?

# Lessons& perspective

 - this looks obsolete and artificial? although real application
 - relevance in topic vs relevance in content
 - MoEs can be fast! We can(and must) waste power for speed. Can we free ride in the future?
 - pdf mining (ToC), KeyBert, summarizatin, Hierachical RAG, CoT, ToT (too much to run)

# Demo


In [25]:
!pip3 freeze > requirements.txt 

## Loading Utils and tweak

In [1]:
from utils import *
dir_path= 'Archive/reprompts'
all_pdf_docs = get_all_docs(dir_path, handler_args={'breaking_pages': False,
                                                    'splitter_args': {'threshold' : 32,'chunk_overlap': 8,}})
vdb = get_vdb(all_pdf_docs)

In [2]:
from functools import partial

search_hard = partial(hard_similarity_search,documents=all_pdf_docs, r=.6, lower = True)
search_vdb = partial(lambda query, vdb, k: vdb.similarity_search(query,k=k), k=3,vdb=vdb)

search_vdb('kim')

[Document(page_content='KARDASHIAN a/k/a KIMBERLY NOEL "KIM" KARDASHIAN, SANDRA BULLOCK, KOURTNEY KARDASHIAN, and', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (4).pdf', 'in_file': 4}),
 Document(page_content='KIMBERLY NOEL "KIM" KARDASHIAN; SANDRA BULLOCK; KOURTNEY KARDASHIAN; JESSE JAMES, DEFENDANTS', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp.pdf', 'in_file': 3}),
 Document(page_content='KARDASHIAN A/K /AKIM BERLY §\nNOEL “KIM” K ARDASHIAN, SANDR A', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (2).pdf', 'in_file': 3})]

In [3]:
from local_light_llm import llm

C:\Users\lhrcp\AppData\Roaming\Python\Python310\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! max_new_tokens is not default parameter.
                max_new_tokens was transferred to model_kwargs.
                Please confirm that max_new_tokens is what you intended.
  warnings.warn(
C:\Users\lhrcp\AppData\Roaming\Python\Python310\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! do_sample is not default parameter.
                do_sample was transferred to model_kwargs.
                Please confirm that do_sample is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from GGUFs\mixtral-8x7b-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                 

In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm= llm, 
                                  chain_type="stuff", 
                                  retriever=vdb.as_retriever(search_kwargs={"k": 3}), 
                                  return_source_documents=True,
                                  verbose = False)

answer_RAG = qa_chain




from advanced_solvers import *
search_tool_func = partial(
        search_metadata_util2,
        docs=all_pdf_docs,
        search_func=vdb.similarity_search,
        # search_args={'k': 6},
        n_args= {'n':64, 'overlap_length':32, 'key':'in_file'},
    )
search_tool = Tool(
    name='case search',
    func=search_tool_func,
    description="""
    Search for the lawsuit case.
    """
)
agent_executor = initialize_agent([search_tool], llm, 
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True,
        handle_parsing_errors=True,
        # memory = memory, 
        )

answer_agent = lambda query: agent_executor({'input': query})
# print(agent_executor.invoke({'input': 'what is Kim accused of'}))

In [20]:
# Define the search and answer functions for each backend
search_functions = {
    "N_gram": search_hard,
    "VDB": search_vdb,
    # Add more backends and their corresponding search functions
}

answer_functions = {
    "RAG": answer_RAG,
    "Search-neighboring_agent": answer_agent,
    # Add more backends and their corresponding answer functions
}

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import pickle
import string

save_dir = "history/"
def convert_punctuations(text, n=50):
    # Create a translation table to map punctuations to underscores
    punct_table = str.maketrans(string.punctuation, '_' * len(string.punctuation))
    # Convert punctuations to underscores and take the first n letters
    converted_text = text.translate(punct_table)[:n]
    return converted_text

def on_search_button_clicked(b):
    query = query_input.value
    save_filename = convert_punctuations(query)
    selected_search_backend = search_backend_dropdown.value
    search_func = search_functions[selected_search_backend]
    searched_docs = search_func(query)
    # Convert searched documents to pandas DataFrame and save as CSV
    df = documents_to_dataframe(searched_docs)
    df.to_csv(save_dir+f"{save_filename}.csv", index=False)
    print(df)
    # Save all objects using a serializer (pickle)
    with open(save_dir+f"{save_filename}.pkl", "wb") as file:
        pickle.dump(searched_docs, file)
    print("Search completed. Results saved.")

def on_answer_button_clicked(b):
    query = query_input.value
    save_filename = convert_punctuations(query)
    selected_search_backend = search_backend_dropdown.value
    selected_answer_backend = answer_backend_dropdown.value
    search_func = search_functions[selected_search_backend]
    answer_func = answer_functions[selected_answer_backend]
    searched_docs = search_func(query)
    result = answer_func(query, searched_docs)
    # Print the result in Jupyter Notebook
    print("Answer:")
    print(result)
    # Save all printed content to a text file
    with open(save_dir+f"{save_filename}.txt", "w") as file:
        file.write(f"Query: {query}\n\n")
        file.write(f"Answer: {result}\n")
    # Save all objects using a serializer (pickle)
    with open(save_dir+f"{save_filename}.pkl", "wb") as file:
        pickle.dump(result, file)
    print("Answer generated. Results saved.")

# Create the UI widgets
query_input = widgets.Text(description="Query:")
search_backend_dropdown = widgets.Dropdown(options=list(search_functions.keys()), description="Search Backend:")
answer_backend_dropdown = widgets.Dropdown(options=list(answer_functions.keys()), description="Answer Backend:")
search_button = widgets.Button(description="Search")
answer_button = widgets.Button(description="Answer")

# Attach click event handlers to the buttons
search_button.on_click(on_search_button_clicked)
answer_button.on_click(on_answer_button_clicked)


Text(value='', description='Query:')

Dropdown(description='Search Backend:', options=('N_gram', 'VDB'), value='N_gram')

Dropdown(description='Answer Backend:', options=('RAG', 'Search-neighboring_agent'), value='RAG')

Button(description='Search', style=ButtonStyle())

Button(description='Answer', style=ButtonStyle())

...

Answer:
{'result': ' The Plaintiff is the person who brings an action against another party. In other words, the Plaintiff is the one who files the lawsuit.', 'source_documents': [Document(page_content='Plaintiff alleges that Defenda nts, who are public figures, have', metadata={'page_number': 1, 'source': 'Archive/reprompts/031141038500.pdf', 'in_file': 10}), Document(page_content='lawsuit, in which the plaintiff, apparently believing federal court is the', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (3).pdf', 'in_file': 21}), Document(page_content='am a plaintiff or petitioner in this case and declare that', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (1).pdf', 'in_file': 8})]}
Answer generated. Results saved.


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4434.88 ms
llama_print_timings:      sample time =       1.90 ms /     6 runs   (    0.32 ms per token,  3162.89 tokens per second)
llama_print_timings: prompt eval time =   19720.26 ms /    83 tokens (  237.59 ms per token,     4.21 tokens per second)
llama_print_timings:        eval time =    1592.59 ms /     5 runs   (  318.52 ms per token,     3.14 tokens per second)
llama_print_timings:       total time =   21348.81 ms /    88 tokens


Answer:
{'result': ' Kim Kardashian', 'source_documents': [Document(page_content='United States District Judge Case', metadata={'page_number': 1, 'source': 'Archive/reprompts/031141038524.pdf', 'in_file': 9}), Document(page_content='GINO ROMANO, Plaintiff, -vs- Case No. A-12-CA-535-SS KIM KARDASHIAN a/k/a KIMBERLY', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (4).pdf', 'in_file': 3}), Document(page_content='due to a fraud on the Court perpetrated by the plaintiff,', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (3).pdf', 'in_file': 26})]}
Answer generated. Results saved.


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4434.88 ms
llama_print_timings:      sample time =       1.62 ms /     6 runs   (    0.27 ms per token,  3710.58 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1850.12 ms /     6 runs   (  308.35 ms per token,     3.24 tokens per second)
llama_print_timings:       total time =    1867.09 ms /     7 tokens


Answer:
{'result': ' Kim Kardashian', 'source_documents': [Document(page_content='United States District Judge Case', metadata={'page_number': 1, 'source': 'Archive/reprompts/031141038524.pdf', 'in_file': 9}), Document(page_content='GINO ROMANO, Plaintiff, -vs- Case No. A-12-CA-535-SS KIM KARDASHIAN a/k/a KIMBERLY', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (4).pdf', 'in_file': 3}), Document(page_content='due to a fraud on the Court perpetrated by the plaintiff,', metadata={'page_number': 1, 'source': 'Archive/reprompts/show_temp (3).pdf', 'in_file': 26})]}
Answer generated. Results saved.


## Let's go

In [24]:

# Display the UI widgets
display(query_input)
display(search_backend_dropdown)
display(answer_backend_dropdown)
display(search_button)
display(answer_button)

Text(value='who won the case', description='Query:')

Dropdown(description='Search Backend:', index=1, options=('N_gram', 'VDB'), value='VDB')

Dropdown(description='Answer Backend:', options=('RAG', 'Search-neighboring_agent'), value='RAG')

Button(description='Search', style=ButtonStyle())

Button(description='Answer', style=ButtonStyle())